In [ ]:
def backProp(batch, model):
    # img1 = utils.load_image("./test_data/tiger.jpeg")
    # img2 = utils.load_image("./test_data/puzzle.jpeg")

    # batch1 = img1.reshape((1, 224, 224, 3))
    # batch2 = img2.reshape((1, 224, 224, 3))

    # batch = np.concatenate((batch1, batch2), 0)
    vgg19_npy_path = './vgg19.npy'
    #batch = np.asarray([img, img])

    # with tf.Session(config=tf.ConfigProto(gpu_options=(tf.GPUOptions(per_process_gpu_memory_fraction=0.7)))) as sess:
    with tf.device('/cpu:0'):
        with tf.Graph().as_default():
            with tf.Session() as sess:
                print(batch.shape)
                images = tf.placeholder("float", batch.shape)
                feed_dict = {images: batch}

                vgg = BallerVgg(vgg19_npy_path)
                optimizer = tf.train.GradientDescentOptimizer(.0001)
                global_step = tf.Variable(0, name='global_step', trainable=False)
                
                labels = tf.to_int64(labels)
                cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=vgg.prob, name='xentropy')
                
                loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
                train_op = optimizer.minimize(loss, global_step=global_step)

                with tf.name_scope("content_vgg"):
                    vgg.build(images)
                
                sess.run(tf.global_variables_initializer())
                for step in xrange(FLAGS.max_steps):
                    feed_dict = fill_feed_dict(data_sets.train,
                               images_placeholder,
                               labels_placeholder)
                    _, loss = sess.run(train_op, feed_dict = feed_dict)
                prob = sess.run(vgg.prob, feed_dict=feed_dict)